In [1]:
#connecting and reading to and from snowflake
import boto3
import snowflake.connector
from pathlib import Path
from snowflake.connector.pandas_tools import write_pandas
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer
from snowflake.connector import DictCursor
import pandas as pd
from snowflake.snowpark import functions as fn
from snowflake.snowpark import types as type
from snowflake.snowpark import DataFrame as df
import snowflake.connector
import pandas as pd
#import streamlit as st
from snowflake.snowpark import Session
#from streamlit_autorefresh import st_autorefresh
import matplotlib.pyplot as plt
import numpy as np
from snowflake.snowpark.functions import col, count, lit, sum as sum_, max as max_, month, year, concat, asc, to_date, floor,dayofyear, dayofweek,dateadd,avg
import re
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark.functions import udf
import datetime as dt

c:\Users\sophie.jones\.virtualenvs\eda_streamlit-675cm0gE\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


In [2]:
# to run : python -m streamlit run EDA.py

def is_outlier(col_name, lower_multiplier=0.3, upper_multiplier=0.3): 
    quartiles = engagement_test.approxQuantile(col_name, [0.25, 0.75]) 
    Q1 = quartiles[0] 
    Q3 = quartiles[1] 
    IQR = Q3 - Q1 
    lower_bound = Q1 - (lower_multiplier * IQR) 
    upper_bound = Q3 + (upper_multiplier * IQR) 
    return (col(col_name) >= lower_bound) & (col(col_name) <= upper_bound) 


# Connect using the Snowflake Snowpark package
#@st.cache_resource
def snowflake_session(connection_parameters) -> Session:
    return Session.builder.configs(connection_parameters).create()

connection_parameters = {
    "user" : "sophie.jones@contractor.itech.media",
    "account" : "gs46004.eu-west-1",
    "role" : "PRD_ANALYST",
    "warehouse" : "PRD_WH",
    "database" : "PRD_DWH",
    "authenticator" : "externalbrowser"
}

main_session = snowflake_session(connection_parameters)
engagement_test = main_session.table('sandbox.sj_engagement_test')

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://netman.okta.com/app/snowflake/exk5idnkinyNtZ20m417/sso/saml?SAMLRequest=lZLdb9owFMX%2Flch7Tux8sA2LUGVl1dDagoC2W9%2Fc5AJeEjvzdQj893P4mLqHVtpb5Jzj3%2FE9d3S1rytvBwalVikJA0Y8ULkupNqk5GF1438mHlqhClFpBSk5AJKr8QhFXTU8a%2B1WLeB3C2g9d5FC3v9ISWsU1wIlciVqQG5zvszubnkUMC4QwViHI2dLgdKxttY2nNKu64IuDrTZ0IgxRtmQOlUv%2BUBeIZr3GY3RVue6ulj27k1vIELKkh7hFI4wPxu%2FSHUawXuUl5MI%2BbfVau7PZ8sV8bLL6661wrYGswSzkzk8LG5PAdAl2GDykbEkgNbv3Oj8MEClu3UlSsh13bTW3Rq4L7qGglZ6I92sppOUNKUs9rv8LvslfqrZk%2FgxUdvF5LsW7GsDj09hDU0x617KoT6UXYI58R4vzUZ9s1PEFqaq79O6IxbFPhv4UbwKYz4Y8jgKkiR8Jt7EhZJK2KPzElqBrYUKdGnFMZtoGvo3NoV9OZCFKqU63NvniNVJ%2BIkiatq3RU4Lw498M%2F7fMYzoa%2Fd59%2B5dHdPJXFcyP3g32tTCvt1WGITHE1n466OUQy1klRWFAUTXWlXp7tqAsG7FrWmB0PGJ%2Bu%2BSj%2F8A&RelayState=61914 to auth

In [26]:
engagement_test.count()

1070809

In [28]:
engagement_test_sample_1 = engagement_test.sample(frac = 0.1)
engagement_test_sample_1.count()

107137

In [29]:
engagement_test_sample_2 = engagement_test.sample(frac = 0.1)
engagement_test_sample_2.count()

106734

In [31]:
#streamlit options for which pings to compare?
#sample method or all?
datalist = engagement_test_sample_1.select(count(col("*")).alias("count_all"),sum_(col("has_engagement_test_30")).alias("sum_30"),((sum_(col("has_engagement_test_30"))/count(col("*")))*100).alias("Engage_rate_30")).collect()
print("Engagement Rate with time on site ping of 30:",datalist[0][1])
#print("Engagement Rate with time on site ping of 90:",datalist[0][4])

Engagement Rate with time on site ping of 30: 74672


In [32]:
datalist = engagement_test_sample_2.select(count(col("*")).alias("count_all"),sum_(col("has_engagement_test_45")).alias("sum_45"),((sum_(col("has_engagement_test_45"))/count(col("*")))*100).alias("Engage_rate_45")).collect()
print("Engagement Rate with time on site ping of 45:",datalist[0][1])

Engagement Rate with time on site ping of 45: 69941


In [16]:
import numpy as np
import pandas as pd
import scipy
import scipy.special

In [17]:
datalist[0][0]

1070809

In [18]:
ar=np.array([[datalist[0][1], (datalist[0][0]-datalist[0][1])],[datalist[0][3], (datalist[0][0]-datalist[0][3])]])    
df=pd.DataFrame(ar, columns=["Engaged", "Not Engaged"])
df.index=["Tos_ping_30", "Tos_ping_90"] 
df 

,Engaged,Not Engaged
Tos_ping_30,745043,325766
Tos_ping_90,607660,463149


In [19]:
df2=df.copy()
df2.loc['Column_Total']= df2.sum(numeric_only=True, axis=0)
df2.loc[:,'Row_Total'] = df2.sum(numeric_only=True, axis=1)
df2

,Engaged,Not Engaged,Row_Total
Tos_ping_30,745043,325766,1070809
Tos_ping_90,607660,463149,1070809
Column_Total,1352703,788915,2141618


In [20]:
n=df2.at["Column_Total", "Row_Total"]  # grand total 

exp=df2.copy()               # create dataframe with expected frequencies
for x in exp.index[0:-1]:
    for y in exp.columns[0:-1]:
        # round expected values to nearest hundredths:
        v= (((df2.at[x, "Row_Total"]) * (df2.at["Column_Total", y])   )   /n ).round(2) 
        exp.at[x,y]=float(v)

exp

,Engaged,Not Engaged,Row_Total
Tos_ping_30,676351.5,394457.5,1070809
Tos_ping_90,676351.5,394457.5,1070809
Column_Total,1352703.0,788915.0,2141618


In [21]:
def p(a): 
    v=(scipy.special.binom(int(df2.iloc[0,2]), a) * scipy.special.binom(int(df2.iloc[1,2]), (int(df2.iloc[2,0])-a)) )/scipy.special.binom(n, int(df2.iloc[2,0]))
    return v

p(1) # if we try "a=1" we get the following probability ... 

0.0

In [22]:
p_observed=p(7) # In our contingency table, a was equal to 7.

p_list=[]
for i in range(int(df2.iloc[0,2])  + 1  ): # calculate p(a) for every possible table we can get given the fixed margins...
#... this ranges from "9 choose 0" to "9 choose 9" so we should get 10 possible tables and their respective probabilities..
    if p(i)<=p_observed:
        p_list.append(p(i))     # append these probabilites to p_list only if <= p_observed
        
p_val=np.sum(p_list) # the sum of this list corresponds to the p-value 
p_val 

C:\Users\sophie.jones\AppData\Local\Temp\ipykernel_20016\2466435755.py:2: RuntimeWarning: invalid value encountered in scalar multiply
  v=(scipy.special.binom(int(df2.iloc[0,2]), a) * scipy.special.binom(int(df2.iloc[1,2]), (int(df2.iloc[2,0])-a)) )/scipy.special.binom(n, int(df2.iloc[2,0]))


KeyboardInterrupt: 

In [70]:
datalist = engagement_test.select(count(col("*")).alias("count_all"),sum_(col("has_engagement_test_30")).alias("sum_30"),((sum_(col("has_engagement_test_30"))/count(col("*")))*100).alias("Engage_rate_30"),sum_(col("has_engagement_test_90")).alias("sum_90"),((sum_(col("has_engagement_test_90"))/count(col("*")))*100).alias("Engage_rate_90")).collect()
print("Engagement Rate with time on site ping of 30:",datalist[0][1])

Engagement Rate with time on site ping of 30: 745043


In [71]:
datalist

[Row(COUNT_ALL=1070809, SUM_30=745043, ENGAGE_RATE_30=Decimal('69.577600'), SUM_90=607660, ENGAGE_RATE_90=Decimal('56.747700'))]

In [72]:
import scipy.stats as stats

data = [[datalist[0][1], (datalist[0][0]-datalist[0][1])],[datalist[0][3], (datalist[0][0]-datalist[0][3])]]
#data = [[datalist[0][1], datalist[0][3]],[(datalist[0][0]-datalist[0][1]), (datalist[0][0]-datalist[0][3])]]
print(data)
oddsratio, pvalue = stats.fisher_exact(data)  
pvalue

[[745043, 325766], [607660, 463149]]


0.0

In [73]:
from scipy.stats import chi2_contingency

stat, p, dof, expected = chi2_contingency(data)
 
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 0.0
Dependent (reject H0)


In [74]:
chi2, p, dof, expected = chi2_contingency(data)

print(f"chi2 statistic:     {chi2:.5g}")
print(f"p-value:            {p:.5g}")
print(f"degrees of freedom: {dof}")
print("expected frequencies:")
print(expected)

chi2 statistic:     37876
p-value:            0
degrees of freedom: 1
expected frequencies:
[[676351.5 394457.5]
 [676351.5 394457.5]]


In [75]:
# Example of calculating the mcnemar test
from statsmodels.stats.contingency_tables import mcnemar

#data = [[datalist[0][1], (datalist[0][0]-datalist[0][1])],[datalist[0][3], (datalist[0][0]-datalist[0][3])]]
#data = [[datalist[0][1], datalist[0][3]],[(datalist[0][0]-datalist[0][1]), (datalist[0][0]-datalist[0][3])]]
print(data)
# define contingency table

# calculate mcnemar test
result = mcnemar(data, exact=True)
# summarize the finding
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
# interpret the p-value
alpha = 0.05
if result.pvalue > alpha:
 print('Same proportions of errors (fail to reject H0)')
else:
 print('Different proportions of errors (reject H0)')

[[745043, 325766], [607660, 463149]]
statistic=325766.000, p-value=0.000
Different proportions of errors (reject H0)
